In [1]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Cipher import PKCS1_OAEP

In [2]:
# Generate RSA Key Pair for the recipient
recipient_key = RSA.generate(2048)  # You can adjust the key size (e.g., 2048 bits)

# Extract the private and public keys for the recipient
recipient_private_key = recipient_key.export_key()
recipient_public_key = recipient_key.publickey().export_key()

# Save the keys to files
with open('recipient_private_key.pem', 'wb') as f:
    f.write(recipient_private_key)

with open('recipient_public_key.pem', 'wb') as f:
    f.write(recipient_public_key)

In [3]:
# Define a message for verification
message = "This is the message to be signed."

# Encrypt the message with the recipient's public key
recipient_public_key = RSA.import_key(open('recipient_public_key.pem', 'rb').read())
cipher = PKCS1_OAEP.new(recipient_public_key)
encrypted_message = cipher.encrypt(message.encode())

# Save the encrypted message to a file
with open('encrypted_message.bin', 'wb') as f:
    f.write(encrypted_message)

# Sign the original message with the sender's private key
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

hash_message = SHA256.new(message.encode())
sender_private_key = RSA.import_key(open('private_key.pem', 'rb').read())
signature = pkcs1_15.new(sender_private_key).sign(hash_message)

# Save the signature to a file
with open('signature.pem', 'wb') as f:
    f.write(signature)

In [4]:
# Sign the original message with the sender's private key
hash_message = SHA256.new(message.encode())
sender_private_key = RSA.import_key(open('private_key.pem', 'rb').read())
signature = pkcs1_15.new(sender_private_key).sign(hash_message)

# Save the signature to a file
with open('signature.pem', 'wb') as f:
    f.write(signature)

In [5]:
# Load the recipient's private key (for decryption)
recipient_private_key = RSA.import_key(open('recipient_private_key.pem', 'rb').read())

# Load the encrypted message
with open('encrypted_message.bin', 'rb') as f:
    encrypted_message = f.read()

# Decrypt the message with the recipient's private key
cipher = PKCS1_OAEP.new(recipient_private_key)
decrypted_message = cipher.decrypt(encrypted_message).decode()

print("Decrypted Message:", decrypted_message)

# Load the sender's public key (for signature verification)
sender_public_key = RSA.import_key(open('public_key.pem', 'rb').read())

# Load the received signature
with open('signature.pem', 'rb') as f:
    received_signature = f.read()

Decrypted Message: This is the message to be signed.


In [6]:
# Verify the signature
hash_received_message = SHA256.new(decrypted_message.encode())

try:
    pkcs1_15.new(sender_public_key).verify(hash_received_message, received_signature)
    print("Signature is valid. The message is original.")
except (ValueError, TypeError):
    print("Signature is invalid. The message may have been tampered with.")

Signature is valid. The message is original.
